In [810]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score,median_absolute_error

In [811]:
df = pd.read_csv('movies_cleaned_2.csv')

In [812]:
df

,release_year,rel_month,rel_day,release_weekday,industry_Hindi,industry_Tamil,industry_Telugu,industry_Kannada,industry_Malayalam,is_pan_india,...,Romantic,Thriller,Other_Genre,weekend1_collection_cr,india_net_cr,worldwide_collection_cr,was_overseas,log_weekend1,log_india_net,log_worldwide
0,2017,12,22,4,1,0,0,0,0,0,...,0,1,0,114.93,339.16,558.00,1,4.752987,5.829416,6.326149
1,2017,12,15,4,1,0,0,0,0,0,...,0,1,0,0.38,0.72,0.88,1,0.322083,0.542324,0.631272
2,2017,12,8,4,1,0,0,0,0,0,...,1,0,0,32.20,80.16,112.30,1,3.502550,4.396423,4.730039
3,2017,12,1,4,1,0,0,0,0,0,...,0,0,1,6.60,10.21,16.60,1,2.028148,2.416806,2.867899
4,2017,12,1,4,1,0,0,0,0,0,...,1,0,1,1.55,2.01,2.75,1,0.936093,1.101940,1.321756
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3364,2025,1,10,4,0,0,0,1,0,0,...,0,1,0,0.11,0.33,0.36,0,0.104360,0.285179,0.307485
3365,2025,1,10,4,0,0,1,0,0,1,...,0,0,0,88.50,131.20,186.28,1,4.494239,4.884316,5.232605
3366,2025,1,3,4,0,0,0,1,0,0,...,0,0,0,0.14,0.26,0.29,0,0.131028,0.231112,0.254642
3367,2025,1,3,4,0,0,0,1,0,0,...,0,0,0,0.02,0.02,0.02,0,0.019803,0.019803,0.019803


In [813]:
# Define the list of regional industry columns
regional_cols = [
    'industry_Hindi', 'industry_Tamil', 'industry_Telugu', 
    'industry_Kannada', 'industry_Malayalam'
]

# Keep rows where at least one regional industry is 1 (Not English)
df = df[df[regional_cols].sum(axis=1) > 0]

In [814]:
X = df.drop(columns=['worldwide_collection_cr', 'india_net_cr', 'log_india_net', 'log_worldwide', 'was_overseas', 'weekend1_collection_cr'])
y = df['log_india_net']

In [815]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Linear Regression

In [816]:
lr = LinearRegression(n_jobs=-1)

In [817]:
lr.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,-1
,positive,False


In [818]:
y_pred = lr.predict(X_test)

In [819]:
print("R2 Score",r2_score(y_test,y_pred))

R2 Score 0.9471194972251678


In [820]:
y_pred_log = lr.predict(X_test)

y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

from sklearn.metrics import mean_absolute_error
print("MAE (₹ Cr):", mean_absolute_error(y_true, y_pred))
print("R2 Score",r2_score(y_test,y_pred_log))
print("Median Absolute Error", median_absolute_error(y_test,y_pred_log))

MAE (₹ Cr): 8.00117271283918
R2 Score 0.9471194972251678
Median Absolute Error 0.14778736143618953


In [821]:
train = df[df['release_year'] <= 2024]
test  = df[df['release_year'] > 2024]

X_train = train[X.columns]
y_train = train['log_india_net']

X_test = test[X.columns]
y_test = test['log_india_net']


In [822]:
lr = LinearRegression(n_jobs=-1)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("R2 Score",r2_score(y_test,y_pred))

R2 Score 0.9396900270253509


In [823]:
y_pred_log = lr.predict(X_test)

y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

print("MAE (₹ Cr):", mean_absolute_error(y_true, y_pred))
print("R2 Score",r2_score(y_test,y_pred_log))
print("Median Absolute Error", median_absolute_error(y_test,y_pred_log))

MAE (₹ Cr): 7.743243939764044
R2 Score 0.9396900270253509
Median Absolute Error 0.19417704972942781


#### Random Forest

In [824]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestRegressor(n_estimators=500, random_state=17, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("R2 Score",r2_score(y_test,y_pred))

R2 Score 0.9764697411390452


In [825]:
y_pred_log = rf.predict(X_test)

y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

print("MAE (₹ Cr):", mean_absolute_error(y_true, y_pred))
print("R2 Score",r2_score(y_test,y_pred_log))
print("Median Absolute Error", median_absolute_error(y_test,y_pred_log))

MAE (₹ Cr): 3.4137105471855542
R2 Score 0.9764697411390452
Median Absolute Error 0.05853579644313474


In [826]:
train = df[df['release_year'] <= 2024]
test  = df[df['release_year'] > 2024]

X_train = train[X.columns]
y_train = train['log_india_net']

X_test = test[X.columns]
y_test = test['log_india_net']


In [827]:
rf = RandomForestRegressor(n_estimators=500, random_state=17, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
print("R2 Score",r2_score(y_test,y_pred))

R2 Score 0.9484941917518025


In [828]:
y_pred_log = rf.predict(X_test)

y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

print("MAE (₹ Cr):", mean_absolute_error(y_true, y_pred))
print("R2 Score",r2_score(y_test,y_pred_log))
print("Median Absolute Error", median_absolute_error(y_test,y_pred_log))

MAE (₹ Cr): 7.906507401691349
R2 Score 0.9484941917518025
Median Absolute Error 0.07805599303123818


#### XGBoost

In [829]:
xgb = XGBRegressor(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    objective='reg:squarederror',
    random_state=42,
    n_jobs=-1
)

xgb.fit(X_train, y_train)


,objective,'reg:squarederror'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,None


In [830]:
y_pred_log = xgb.predict(X_test)
y_pred = np.expm1(y_pred_log)
y_true = np.expm1(y_test)

print("MAE (₹ Cr):", mean_absolute_error(y_true, y_pred))
print("R2:", r2_score(y_test, y_pred_log))
print("Median Absolute Error", median_absolute_error(y_test,y_pred_log))

MAE (₹ Cr): 7.376923919922004
R2: 0.9520156859194583
Median Absolute Error 0.08563857594940422


I evaluated linear regression, Random Forest, and XGBoost models using a temporal split with 2024 onward movies held out. While XGBoost achieved the lowest MAE (₹7.5 Cr), Random Forest offered comparable performance with better interpretability and stability, and was selected as the final model.”

In [831]:
X

,release_year,rel_month,rel_day,release_weekday,industry_Hindi,industry_Tamil,industry_Telugu,industry_Kannada,industry_Malayalam,is_pan_india,...,Comedy,Crime,Drama,Family,Horror,Mystery,Romantic,Thriller,Other_Genre,log_weekend1
0,2017,12,22,4,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,4.752987
1,2017,12,15,4,1,0,0,0,0,0,...,0,1,0,0,0,0,0,1,0,0.322083
2,2017,12,8,4,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,3.502550
3,2017,12,1,4,1,0,0,0,0,0,...,1,0,1,0,0,0,0,0,1,2.028148
4,2017,12,1,4,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0.936093
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3364,2025,1,10,4,0,0,0,1,0,0,...,0,1,1,0,0,0,0,1,0,0.104360
3365,2025,1,10,4,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,4.494239
3366,2025,1,3,4,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0.131028
3367,2025,1,3,4,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0.019803


In [832]:
y

0       5.829416
1       0.542324
2       4.396423
3       2.416806
4       1.101940
          ...   
3364    0.285179
3365    4.884316
3366    0.231112
3367    0.019803
3368    0.039221
Name: log_india_net, Length: 3174, dtype: float64

## Model 2 (World Wide)

In [833]:
df['predicted_log_india_net'] = rf.predict(X)
df['predicted_india_net'] = np.expm1(df['predicted_log_india_net'])

C:\Users\bsais\AppData\Local\Temp\ipykernel_6128\1725120597.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_log_india_net'] = rf.predict(X)
C:\Users\bsais\AppData\Local\Temp\ipykernel_6128\1725120597.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_india_net'] = np.expm1(df['predicted_log_india_net'])


In [834]:
df.sample(5)

,release_year,rel_month,rel_day,release_weekday,industry_Hindi,industry_Tamil,industry_Telugu,industry_Kannada,industry_Malayalam,is_pan_india,...,Other_Genre,weekend1_collection_cr,india_net_cr,worldwide_collection_cr,was_overseas,log_weekend1,log_india_net,log_worldwide,predicted_log_india_net,predicted_india_net
138,2020,11,14,5,0,1,0,0,0,0,...,0,1.70,3.17,3.40,0,0.993252,1.427916,1.481605,1.413673,3.111029
131,2020,1,9,3,0,1,0,0,0,0,...,0,85.00,149.60,247.80,1,4.454347,5.014627,5.516649,5.006589,148.394355
879,2022,8,12,4,0,0,0,1,0,0,...,0,13.00,32.13,38.50,1,2.639057,3.500439,3.676301,3.435800,30.056262
1170,2024,7,26,4,1,0,0,0,0,0,...,0,0.07,0.10,0.13,0,0.067659,0.095310,0.122218,0.132961,0.142206
1710,2024,8,9,4,0,1,0,0,0,0,...,0,0.00,0.04,0.05,0,0.000000,0.039221,0.048790,0.359662,0.432845


In [835]:
# Define the list of regional industry columns
regional_cols = [
    'industry_Hindi', 'industry_Tamil', 'industry_Telugu', 
    'industry_Kannada', 'industry_Malayalam'
]

# Keep rows where at least one regional industry is 1 (Not English)
df = df[df[regional_cols].sum(axis=1) > 0]

In [836]:
# Industry as English
df2 = df[df['industry_Hindi'] == 0]
df2 = df2[df2['industry_Tamil'] == 0]
df2 = df2[df2['industry_Telugu'] == 0]
df2 = df2[df2['industry_Kannada'] == 0]
df2 = df2[df2['industry_Malayalam'] == 0]

df2.shape

(0, 38)

In [837]:
df = df[df['was_overseas'] == 1]
df.sample(5)

,release_year,rel_month,rel_day,release_weekday,industry_Hindi,industry_Tamil,industry_Telugu,industry_Kannada,industry_Malayalam,is_pan_india,...,Other_Genre,weekend1_collection_cr,india_net_cr,worldwide_collection_cr,was_overseas,log_weekend1,log_india_net,log_worldwide,predicted_log_india_net,predicted_india_net
804,2022,6,3,4,1,0,0,0,0,0,...,1,39.40,68.25,90.55,1,3.698830,4.237723,4.516885,4.241329,68.500155
773,2022,10,25,1,1,0,0,0,0,0,...,1,56.00,74.70,96.74,1,4.043051,4.326778,4.582311,4.337728,75.533427
2637,2025,3,14,4,0,0,1,0,0,0,...,0,3.85,7.10,12.15,1,1.578979,2.091864,2.576422,2.285472,8.830323
19,2017,9,15,4,1,0,0,0,0,0,...,1,8.42,11.20,16.95,1,2.242835,2.501436,2.887590,2.633725,12.925539
1450,2024,6,27,3,0,0,1,0,0,1,...,1,309.00,646.31,1042.25,1,5.736572,6.472825,6.950096,6.474900,647.654634


In [838]:
df['log_residual'] = (
    df['log_worldwide'] - df['predicted_log_india_net']
)

In [ ]:
train_m2 = df[df['release_year'] <= 2024]
test_m2  = df[df['release_year'] > 2024]

X2_train = train_m2.drop(columns=['worldwide_collection_cr', 'india_net_cr', 'log_india_net', 'log_worldwide', 'weekend1_collection_cr', 'predicted_india_net','was_overseas','log_residual'])
y2_train = train_m2['log_residual']

X2_test = test_m2.drop(columns=['worldwide_collection_cr', 'india_net_cr', 'log_india_net', 'log_worldwide', 'weekend1_collection_cr', 'predicted_india_net','was_overseas','log_residual'])
y2_test = test_m2['log_residual']


In [840]:
X2_train

,release_year,rel_month,rel_day,release_weekday,industry_Hindi,industry_Tamil,industry_Telugu,industry_Kannada,industry_Malayalam,is_pan_india,...,Drama,Family,Horror,Mystery,Romantic,Thriller,Other_Genre,log_weekend1,predicted_log_india_net,log_residual
0,2017,12,22,4,1,0,0,0,0,0,...,0,0,0,0,0,1,0,4.752987,5.709389,0.616761
1,2017,12,15,4,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0.322083,0.521358,0.109913
2,2017,12,8,4,1,0,0,0,0,0,...,1,0,0,0,1,0,0,3.502550,4.333461,0.396578
3,2017,12,1,4,1,0,0,0,0,0,...,1,0,0,0,0,0,1,2.028148,2.543813,0.324086
4,2017,12,1,4,1,0,0,0,0,0,...,0,0,0,0,1,0,1,0.936093,1.186973,0.134783
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2148,2024,4,11,3,1,0,0,0,0,1,...,0,0,0,0,0,0,0,3.732896,4.203879,0.519074
2155,2024,3,28,3,0,0,0,0,1,1,...,1,0,0,0,0,0,0,3.443618,4.456781,0.609730
2191,2024,2,15,3,0,0,0,0,1,1,...,0,0,1,0,0,1,0,2.700690,3.338007,0.744940
2212,2024,1,12,4,0,1,0,0,0,1,...,1,0,0,0,0,0,1,3.279030,3.918810,0.412976


In [841]:
X2_test

,release_year,rel_month,rel_day,release_weekday,industry_Hindi,industry_Tamil,industry_Telugu,industry_Kannada,industry_Malayalam,is_pan_india,...,Drama,Family,Horror,Mystery,Romantic,Thriller,Other_Genre,log_weekend1,predicted_log_india_net,log_residual
2215,2025,12,25,3,0,0,0,0,1,1,...,1,0,0,0,0,0,1,0.824175,1.340612,-0.190040
2217,2025,12,25,3,1,0,0,0,0,0,...,1,0,0,0,1,0,0,3.198673,3.817380,0.064184
2220,2025,12,12,4,0,0,1,0,0,1,...,1,0,0,0,0,0,0,3.990834,4.585708,0.238597
2222,2025,12,12,4,1,0,0,0,0,0,...,0,0,0,0,1,0,0,2.110213,2.978280,-0.131628
2225,2025,12,5,4,1,0,0,0,0,0,...,0,0,0,0,0,1,1,4.644391,5.361853,1.740975
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257,2025,6,27,4,0,1,0,0,0,1,...,1,0,0,0,0,0,1,3.194583,3.910546,-0.092834
3259,2025,6,20,4,0,0,1,0,0,1,...,1,0,0,0,0,1,1,3.903991,4.591317,0.343876
3286,2025,5,1,3,0,0,1,0,0,1,...,0,0,0,1,0,1,0,3.973118,4.322019,0.469465
3305,2025,3,27,3,0,0,0,0,1,1,...,0,0,0,0,0,1,0,4.094345,4.406521,1.183757


#### Random Forest

In [842]:
rf = RandomForestRegressor(n_estimators=500, random_state=17, n_jobs=-1)
rf.fit(X2_train, y2_train)
y2_pred = rf.predict(X2_test)
print("R2 Score",r2_score(y2_test,y2_pred))

R2 Score 0.6683964841473157


In [843]:
residual_pred = rf.predict(X2_test)
log_worldwide_pred = (
    test_m2['predicted_log_india_net'].values + residual_pred
)
worldwide_pred = np.expm1(log_worldwide_pred)
worldwide_true = test_m2['worldwide_collection_cr'].values


In [844]:
print("Worldwide MAE (₹ Cr):",
      mean_absolute_error(worldwide_true, worldwide_pred))

print("Worldwide Median AE (₹ Cr):",
      median_absolute_error(worldwide_true, worldwide_pred))

print("Worldwide R2 (log-space):",
      r2_score(np.log1p(worldwide_true), log_worldwide_pred))


Worldwide MAE (₹ Cr): 167.7141481380168
Worldwide Median AE (₹ Cr): 0.11412274262329092
Worldwide R2 (log-space): 0.8631543331696214


#### XGBoost

In [860]:
xgb = XGBRegressor(
    n_estimators=500,
    learning_rate=0.1,
    subsample=0.8,
    random_state=42,
    n_jobs=-1
)

xgb.fit(X2_train, y2_train)
y2_pred = xgb.predict(X2_test)
print("R2 Score",r2_score(y2_test,y2_pred))

R2 Score 0.6978661760705578


In [861]:
residual_pred = xgb.predict(X2_test)
log_worldwide_pred = (
    test_m2['predicted_log_india_net'].values + residual_pred
)
worldwide_pred = np.expm1(log_worldwide_pred)
worldwide_true = test_m2['worldwide_collection_cr'].values


In [862]:
print("Worldwide MAE (₹ Cr):",
      mean_absolute_error(worldwide_true, worldwide_pred))

print("Worldwide Median AE (₹ Cr):",
      median_absolute_error(worldwide_true, worldwide_pred))

print("Worldwide R2 (log-space):",
      r2_score(np.log1p(worldwide_true), log_worldwide_pred))


Worldwide MAE (₹ Cr): 172.35898289899984
Worldwide Median AE (₹ Cr): 0.23299102760134538
Worldwide R2 (log-space): 0.8753158436775905


In [863]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1078 entries, 0 to 3365
Data columns (total 39 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   release_year             1078 non-null   int64  
 1   rel_month                1078 non-null   int64  
 2   rel_day                  1078 non-null   int64  
 3   release_weekday          1078 non-null   int64  
 4   industry_Hindi           1078 non-null   int64  
 5   industry_Tamil           1078 non-null   int64  
 6   industry_Telugu          1078 non-null   int64  
 7   industry_Kannada         1078 non-null   int64  
 8   industry_Malayalam       1078 non-null   int64  
 9   is_pan_india             1078 non-null   int64  
 10  num_languages            1078 non-null   int64  
 11  English                  1078 non-null   int64  
 12  Hindi                    1078 non-null   int64  
 13  Tamil                    1078 non-null   int64  
 14  Telugu                   1078

In [ ]:
X_train